# Creating a pilot pipline for Gene Story

![Image description](images/Concept_one.png)

## 1. A python script that take a list of genes from txt file and converting it to a list

## 2. A script that take list and proccess it

- It takes the list.
- First name from the list.
- Search Pubmed for all (n=3) original papers (with an impact factor more than 5) between date A and date B for the key word
- And Save the search reasult containing:
    - Title, Authors, Abstract, Keywords, Journal name, Year
- Save it in a database.

## 3. Creating a Database

In [15]:
import pandas as pd
from Bio import Entrez
import sqlite3
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv('/Users/probio/Documents/codes/Publication2Knowledge/.env')

# Set your email and API key for Entrez
Entrez.email = api_key = os.getenv('my_email')  # Replace with your email
Entrez.api_key = api_key = os.getenv('my_api_Key')  # Replace with your API key (optional)

def search_pubmed(query, mindate, maxdate):
    handle = Entrez.esearch(db='pubmed', sort='pub date', retmax='10', retmode='xml', term=query, mindate=mindate, maxdate=maxdate)
    results = Entrez.read(handle)
    handle.close()
    return results['IdList']

def fetch_details(id_list):
    if not id_list:
        return []  # Return an empty list if id_list is empty
    ids = ','.join(id_list)
    handle = Entrez.efetch(db='pubmed', id=ids, retmode='xml')
    results = Entrez.read(handle, validate=False)  # Disable DTD validation
    handle.close()
    return results


def initialize_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            gene TEXT,
            title TEXT,
            abstract TEXT,
            authors TEXT,
            journal TEXT,
            year TEXT,
            keywords TEXT
        );
    """)
    conn.commit()
    conn.close()


def parse_article(article):
    article_data = article['MedlineCitation']['Article']
    title = article_data.get('ArticleTitle')
    abstract = article_data.get('Abstract', {}).get('AbstractText', [])
    abstract_text = ' '.join(abstract) if abstract else ''
    authors = [author['ForeName'] + ' ' + author['LastName'] for author in article_data.get('AuthorList', [])]
    journal = article_data.get('Journal', {}).get('Title')
    year = article_data.get('Journal', {}).get('JournalIssue', {}).get('PubDate', {}).get('Year')
    keywords = article.get('KeywordList', [])
    return title, abstract_text, ', '.join(authors), journal, year, ', '.join(keywords)

def main(genes, mindate, maxdate, db_path):
    initialize_database(db_path)  # Initialize the database and create the table if it doesn't exist
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    for gene in genes:
        try:
            print(f"Processing {gene}...")
            id_list = search_pubmed(gene, mindate, maxdate)
            papers = fetch_details(id_list)
            if papers:
                for paper in papers['PubmedArticle']:
                    title, abstract, authors, journal, year, keywords = parse_article(paper)
                    cursor.execute("INSERT INTO articles (gene, title, abstract, authors, journal, year, keywords) VALUES (?, ?, ?, ?, ?, ?, ?)", 
                                   (gene, title, abstract, authors, journal, year, keywords))
                conn.commit()
            else:
                print(f"No results found for {gene}")
        except Exception as e:
            print(f"An error occurred while processing {gene}: {e}")
    conn.close()

# Example usage
genes = ['53BP1 AND RIF1']  # Replace with your list of genes
mindate = '2018/01/01'
maxdate = '2023/01/01'
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/publication.db'  # Replace with your database path

main(genes, mindate, maxdate, db_path)


Processing 53BP1 AND RIF1...


### Checking entire database

In [21]:
# Checking the database

import sqlite3

def get_all_articles(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Prepare and execute the SQL query to select all records
    query = "SELECT * FROM articles"
    cursor.execute(query)

    # Fetch all records
    results = cursor.fetchall()

    # Close the database connection
    conn.close()

    return results

# Example usage
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/publication.db'  # Replace with your database path
all_articles = get_all_articles(db_path)

# Print the results
for article in all_articles:
    print(article)


(1, 'RIF1 AND 53BP1', 'Reshaping Chromatin Architecture around DNA Breaks.', 'DNA double-strand breaks (DSBs) elicit major chromatin changes. Using super-resolution microscopy in human cells, Ochs et\xa0al. unveil that the DSB response protein 53BP1 and its effector RIF1 organize DSB-flanking chromatin into circular micro-domains. These structures control the spatial distribution of DSB repair factors safeguarding genome integrity.', 'Pierre Caron, Sophie E Polo', 'Trends in biochemical sciences', '2020', '', 'DNA Double-Strand Breaks -> Chromatin Changes (53BP1 & RIF1 Organization) -> Circular Micro-Domains -> Spatial Control of DSB Repair Factors -> Safeguard Genome Integrity')
(2, 'RIF1 AND 53BP1', '53BP1/RIF1 signaling promotes cell survival after multifractionated radiotherapy.', 'Multifractionated irradiation is the mainstay of radiation treatment in cancer therapy. Yet, little is known about the cellular DNA repair processes that take place between radiation fractions, even thou

## Pulling out abstract from database

In [22]:
import sqlite3

def get_abstract_by_id(db_path, article_id):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Prepare and execute the SQL query
    query = f"SELECT abstract FROM articles WHERE id = {article_id}"
    cursor.execute(query)

    # Fetch the first record
    result = cursor.fetchone()

    # Close the database connection
    conn.close()

    # Return the abstract if a record was found, else return None
    return result[0] if result else None

# Example usage
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/Publication.db'  # Replace with your database path
article_id = 3  # The ID of the article
abstract = get_abstract_by_id(db_path, article_id)

if abstract:
    print("Abstract:", abstract)
else:
    print("No record found for the given ID.")


Abstract: To summarize recently discovered PARP inhibitor resistance mechanisms and highlight the clinical relevance of these findings to date. A predominant mechanism of acquired PARP inhibitor resistance in homologous recombination-deficient cancers is the acquisition of homologous recombination proficiency as a consequence of secondary genetic or epigenetic events, such as secondary mutations in BRCA1 or BRCA2, or reversal of BRCA1 promoter methylation that restores homologous recombination and leads to PARP inhibitor resistance. Multiple other potential mechanisms of acquired resistance to PARP inhibitors including loss of DNA end resection inhibition (53BP1/REV7/RIF1/Sheldin) or DNA replication fork protection (PTIP/EZH2), but also increased drug efflux or induction of a reversible senescent or mesenchymal cell state have been described in ovarian cancer models. However, only few of these mechanisms have been identified in clinical samples. Multiple adaptive responses following PA

In [23]:
Input_text = abstract

### A python script that takes abstract send it to an LLM with pre-defined system task to create a summay.

- Generated summary should be stored in the generated database
    - Title, Authors, Abstract, Keywords, Journal name, Year, summary (by LLM)


Pre-prompt:

V6 *********

"You are a scientist which will use following guideline to create a summary with a specific format under following Guideline. Here's a guideline to assist you in converting an abstract into a similar concise summary: 1. Identify Primary Pathway Components: Focus on pinpointing the main proteins, molecules, or complexes mentioned in the abstract. Emphasize elements that are central to the pathway or process being described. 2. Establish Linear Relationships: Determine how these components are linearly related to each other. This involves identifying which components act upstream or downstream in a pathway. Look for terms like 'upstream,' 'downstream,' 'activates,' 'inhibits,' or 'interacts with'. 3. Construct a Pathway Diagram in Text: Convert the relationships into a simplified, linear pathway diagram using arrows ('--->') to indicate direct influence or flow, and a bar ('---|') to denote inhibition or blockage. The goal is to create a visual representation in text format. 4. Highlight Contextual Conditions: Include specific conditions or contexts (e.g., cell type, genetic status) in which the pathway operates. These should be mentioned at the end of the pathway diagram to provide clarity on the specific scenario in which these interactions occur. 5. Simplify and Focus on Core Elements: Strip away any secondary details that do not contribute to the main pathway structure. Focus on the core elements that are essential for understanding the primary process or interaction. 6. In your response do not put the original abstract in your answer. The response should has a structure as follow: 53BP1--->RIF1--->Shieldin--->CCAR2/DBC1 ---| DNA end resection (in BRCA1 deficient context). 7. Do not add any extra explanation before or after expected response."



"You are a scientist which will use following guideline to create a summary with a specific format under following Guideline. Here's a guideline to assist you in converting an abstract into a similar concise summary: 1. Identify Primary Pathway Components: Focus on pinpointing the main proteins, molecules, complexes or componds mentioned in the abstract. Emphasize elements that are central to the pathway or process being described. 2. Establish Linear Relationships: Determine how these components are linearly related to each other. This involves identifying which components act upstream or downstream in a pathway. Look for terms like 'upstream,' 'downstream,' 'activates,' 'inhibits,' or 'interacts with'. 3. Construct a Pathway Diagram in Text: Convert the relationships into a simplified, linear pathway diagram using arrows ('--->') to indicate direct influence or flow, and a bar ('---|') to denote inhibition or blockage. The goal is to create a visual representation in text format. 4. Highlight Contextual Conditions: Include specific conditions or contexts (e.g., cell type, genetic status) in which the pathway operates. These should be mentioned at the end of the pathway diagram to provide clarity on the specific scenario in which these interactions occur. 5. Simplify and Focus on Core Elements: Strip away any secondary details that do not contribute to the main pathway structure. Focus on the core elements that are essential for understanding the primary process or interaction. 6. In your response do not put the original abstract in your answer. The response should has a structure as follow: CompondA--->or---|ProteinB--->or---|ProteinC--->or---|ProteinD/E --->or---| X process (in Y deficient context). 7. Do not add any extra explanation before or after expected response."



"You are a scientist which will use following guideline to create a summary with a specific format under following Guideline. Here's a guideline to assist you in converting an abstract into a similar concise summary: 1. Identify Primary Pathway Components: Focus on pinpointing the main proteins, molecules, complexes or componds mentioned in the abstract. Emphasize elements that are central to the pathway or process being described. 2. Establish Linear Relationships: Determine how these components are linearly related to each other. This involves identifying which components act upstream or downstream in a pathway. Look for terms like 'upstream,' 'downstream,' 'activates,' 'inhibits,' or 'interacts with'. 3. Construct a Pathway Diagram in Text: Convert the relationships into a simplified, linear pathway diagram using arrows ('--->') to indicate direct influence or flow, and a bar ('---|') to denote inhibition or blockage. The goal is to create a visual representation in text format. 4. Highlight Contextual Conditions: Include specific conditions or contexts (e.g., cell type, genetic status) in which the pathway operates. These should be mentioned at the end of the pathway diagram to provide clarity on the specific scenario in which these interactions occur. 5. Simplify and Focus on Core Elements: Strip away any secondary details that do not contribute to the main pathway structure. Focus on the core elements that are essential for understanding the primary process or interaction. 6. In your response do not put the original abstract in your answer. The response should has a structure as follow: CompondA--->or---|ProteinB--->or---|ProteinC--->or---|ProteinD/E --->or---| X process (in Y deficient context). 7. Do not add any extra explanation before or after expected response."


New version:
"Read a scientific text and identify important elements like genes, proteins, and drugs, making sure each is no more than two words long. Understand how these elements are connected, focusing on whether they inhibit (like 'inhibit', 'degrade', 'disrupt', 'interfere', 'suppress', 'blockade') or promote (like 'activate', 'promote', 'recruit', 'induce', 'enhance', 'facilitate') each other, using only two words to describe these connections. Instead of creating a graph, simply list the connections in a format like 'A inhibits B' or 'C promotes D', where A, B, C, and D are the key elements you identified."


In [30]:
from openai import OpenAI

client= OpenAI(base_url="http://localhost:5001/v1", api_key="not-needed")

try:
    completion = client.chat.completions.create(
        model="marcoroni",  # Ensure the model name is correct. It should be a valid model name from OpenAI.
        messages=[
            {
                "role": "system", 
                "content": ("You are a scientist which will use following guideline to create a summary with a specific format under following Guideline. Here's a guideline to assist you in converting an abstract into a similar concise summary: 1. Identify Primary Pathway Components: Focus on pinpointing the main proteins, molecules, complexes or componds mentioned in the abstract. Emphasize elements that are central to the pathway or process being described. 2. Establish Linear Relationships: Determine how these components are linearly related to each other. This involves identifying which components act upstream or downstream in a pathway. Look for terms like 'upstream,' 'downstream,' 'activates,' 'inhibits,' or 'interacts with'. 3. Construct a Pathway Diagram in Text: Convert the relationships into a simplified, linear pathway diagram using arrows ('--->') to indicate direct influence or flow, and a bar ('---|') to denote inhibition or blockage. The goal is to create a visual representation in text format. 4. Highlight Contextual Conditions: Include specific conditions or contexts (e.g., cell type, genetic status) in which the pathway operates. These should be mentioned at the end of the pathway diagram to provide clarity on the specific scenario in which these interactions occur. 5. Simplify and Focus on Core Elements: Strip away any secondary details that do not contribute to the main pathway structure. Focus on the core elements that are essential for understanding the primary process or interaction. 6. In your response do not put the original abstract in your answer. The response should has a structure as follow: CompondA--->or---|ProteinB--->or---|ProteinC--->or---|ProteinD/E --->or---| X process (in Y deficient context). 7. Do not add any extra explanation before or after expected response.")
             },
            {
                "role": "user", 
                "content": ("Mesenchymal stem cells (MSCs) have been widely used in tissue regeneration and stem cell therapy and are currently being tested in numerous clinical trials. Senescence-related changes in MSC properties have attracted considerable attention. Senescent MSCs exhibit a compromised potential for proliferation; senescence acts as a stress response that prevents the proliferation of dysfunctional cells by inducing an irreversible cell cycle arrest. Here, we established a senescent MSC model using senescence-associated β-galactosidase, proliferation, and cell cycle assays. We further identified novel biomarker candidates for old, senescent tonsil-derived MSCs (TMSCs) using transcriptomics. A plot of the cellular senescence pathway showed cyclin-dependent kinase 1 (CDK1; +8-fold) and CDK2 (+2-fold), and transforming growth factor beta 2 (TGFB2; +2-fold) showed significantly higher expression in old TMSCs than in young TMSCs. The CDK family was shown to be related to cell cycle and proliferation, as confirmed by quantitative RT-PCR. As replicative senescence of TMSCs, the gene and protein expression of CDK1 was significantly increased, which was further validated by inhibiting CDK1 using an inhibitor and siRNA. Taken together, we suggest that the CDK1 can be used as a selective senescence biomarker of MSCs and broaden the research criteria for senescent mechanisms.")
            },
         ],
        temperature=0.5,
    )
except Exception as e:
    print(f"An error occurred: {e}")
    # Handle the error appropriately

# Ensure that 'text' contains the string content of the message
text = completion.choices[0].message.content if completion else ""
print(text)

 Senescent MSCs (Mesenchymal stem cells) ---> Cyclin-dependent kinase 1 (CDK1) upregulation ---> Cell cycle arrest in replicative senescence context.


In [26]:
from openai import OpenAI

client= OpenAI(base_url="http://localhost:5001/v1", api_key="not-needed")

try:
    completion = client.chat.completions.create(
        model="marcoroni",  # Ensure the model name is correct. It should be a valid model name from OpenAI.
        messages=[
            {
                "role": "system", 
                "content": ("You are a scientist which will use following guideline to create a summary with a specific format under following Guideline. Here's a guideline to assist you in converting an abstract into a similar concise summary: 1. Identify Primary Pathway Components: Focus on pinpointing the main proteins, molecules, complexes or componds mentioned in the abstract. Emphasize elements that are central to the pathway or process being described. 2. Establish Linear Relationships: Determine how these components are linearly related to each other. This involves identifying which components act upstream or downstream in a pathway. Look for terms like 'upstream,' 'downstream,' 'activates,' 'inhibits,' or 'interacts with'. 3. Construct a Pathway Diagram in Text: Convert the relationships into a simplified, linear pathway diagram using arrows ('--->') to indicate direct influence or flow, and a bar ('---|') to denote inhibition or blockage. The goal is to create a visual representation in text format. 4. Highlight Contextual Conditions: Include specific conditions or contexts (e.g., cell type, genetic status) in which the pathway operates. These should be mentioned at the end of the pathway diagram to provide clarity on the specific scenario in which these interactions occur. 5. Simplify and Focus on Core Elements: Strip away any secondary details that do not contribute to the main pathway structure. Focus on the core elements that are essential for understanding the primary process or interaction. 6. In your response do not put the original abstract in your answer. The response should has a structure as follow: CompondA--->ProteinB--->ProteinC--->ProteinD/E ---| X process (in Y deficient context). 7. Do not add any extra explanation before or after expected response.")
             },
            {
                "role": "user", 
                "content": (Input_text)
            },
         ],
        temperature=0.4,
    )
except Exception as e:
    print(f"An error occurred: {e}")
    # Handle the error appropriately

# Ensure that 'text' contains the string content of the message
text = completion.choices[0].message.content if completion else ""
print(text)

 53BP1--->RIF1--->Shieldin--->CCAR2/DBC1 ---| DNA end resection (in BRCA1 deficient context). Multiple other resistance mechanisms include loss of DNA fork protection or increased drug efflux, but have limited clinical evidence. Adaptive responses identified post-treatment; further research required for clinical resistance understanding and novel combination therapy development.


Abstract: Chromosomal instability (CIN) is a hallmark of cancer and comprises structural CIN (S-CIN) and numerical or whole chromosomal CIN (W-CIN). Recent work indicated that replication stress (RS), known to contribute to S-CIN, also affects mitotic chromosome segregation, possibly explaining the common co-existence of S-CIN and W-CIN in human cancer. Here, we show that RS-induced increased origin firing is sufficient to trigger W-CIN in human cancer cells. We discovered that overexpression of origin firing genes, including GINS1 and CDC45, correlates with W-CIN in human cancer specimens and causes W-CIN in otherwise chromosomally stable human cells. Furthermore, modulation of the ATR-CDK1-RIF1 axis increases the number of firing origins and leads to W-CIN. Importantly, chromosome missegregation upon additional origin firing is mediated by increased mitotic microtubule growth rates, a mitotic defect prevalent in chromosomally unstable cancer cells. Thus, our study identifies increased replication origin firing as a cancer-relevant trigger for chromosomal instability.

Marcoroni 5Q ### 13.7s
GINS1--->CDC45--->Origin Firing --->ATR-CDK1-RIF1 Axis ----> Increased Origin Firing --->Microtubule Growth Rates ---| Chromosome Missegregation (in Chromosomally Unstable Cancer Cells).

GINS1/CDC45--->Origin Firing--->ATR-CDK1-RIF1 Axis---| Increased Replication Origin Firing (in Cancer Context) --->Mitotic Microtubule Growth Rate --->Chromosome Missegregation --->W-CIN.

Marcoroni 3Q ### 13.4s
GINS1/CDC45 ---> Origin Firing ---> ATR-CDK1-RIF1 axis ---> Increased microtubule growth rates (in chromosomally unstable context) --> S-CIN & W-CIN.

GINS1/CDC45 ---> Origin Firing ---> ATR-CDK1-RIF1 axis ---> Increased firing origins ---> Mitotic microtubule growth rates (increased) ---> Chromosome missegregation ---> W-CIN (in chromosomally unstable cancer cells).

Mistral instruct v0 2 7B Q5_K_M ### 13.1s
53BP1--->ATR (activates)--->CDK1--->RIF1--->Shieldin--->ORC/MCM2-7 (upon DNA damage)---->GINS1, CDC45 (overexpression) ------> Increased origin firing --------> Mitotic microtubule growth rates increase ---> Chromosome missegregation ---> W-CIN (in chromosomally unstable cancer cells context).

53BP1--->ATR--->CDK1--->Cdc45/GINS1--->Origin Firing ----> Mitotic Microtubule Growth Rates Increase ---> Chromosome Missegregation ---> W-CIN (in cancer context).

phi-2 Q8 ### 8.0s
This paper discusses the relationship between replication stress and chromosome segregation in human cancer cells. The authors found that overexpression of certain genes can lead to an increase in the number of origins during replication, which can cause chromosome missegregation. This is particularly relevant in cancer cells, where chromosomal instability is a common feature.

llama 2 chat 7B 5Q_K_M ### 10.7s
53BP1--->RIF1--->Shieldin--->CCAR2/DBC1 ---| DNA end resection (in BRCA1 deficient context).
53BP1--->RIF1--->Shieldin--->CCAR2/DBC1 ---| DNA end resection (in BRCA1 deficient context).

solar 19 instruct v1 9 34B Q5_K_M ### 21.5 sec
CDC45--->GINS1--->Increased Origin Firing (in human cancer cells) 
ATR--->CDK1--->RIF1--->Increased Origin Firing 
---| Chromosome Segregation Defects (mitotic microtubule growth rate increase in chromosomally unstable context)

Catmarcaroni slerp 7B Q5_K_M ### 12.6
GINS1--->CDC45--->Origin Firing (ATR-CDK1-RIF1 axis)---| Increased Replication Origin Firing ---> Chromosome Missegregation ---> W-CIN (in cancer context).


sauerkraut una solar instruct 34B Q5_K_M ### 16.1

CDC45/GINS1--->ATR--->CDK1--->RIF1 --->Increased microtubule growth rates (in human cancer cells).
GINS1/CDC45--->ATR--->CDK1--->RIF1 --->Increased microtubule growth rates (in human cancer context).

## Store the summary in the database associated with the given article ID

In [12]:
#########################################################################################
# Store the summary in the database associated with the given article ID
#########################################################################################

def store_summary(db_path, article_id, summary):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Add a new column for the summary if it does not exist
    cursor.execute("ALTER TABLE articles ADD COLUMN summary TEXT")
    conn.commit()

    # Prepare and execute the SQL query to update the summary for the given article ID
    query = f"UPDATE articles SET summary = ? WHERE id = ?"
    cursor.execute(query, (summary, article_id))

    # Commit the changes and close the database connection
    conn.commit()
    conn.close()

# Example usage
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/Publication.db'  # Replace with your database path
article_id = 1  # The ID of the article for which the summary was generated
summary = text  # The summary text generated by the LLM

store_summary(db_path, article_id, summary)



### Pipline goes back to the list and takes next gene in the list.

#### Printing summary

In [4]:
import sqlite3

def summarize_text(text, max_length=100):
    """
    Generate a simple summary of the text by truncating it.
    :param text: The text to summarize.
    :param max_length: The maximum length of the summary.
    :return: A summarized version of the text.
    """
    return text if len(text) <= max_length else text[:max_length] + '...'

def get_abstract_by_id(db_path, article_id):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Prepare and execute the SQL query
    query = f"SELECT summary FROM articles WHERE id = {article_id}"
    cursor.execute(query)

    # Fetch the first record
    result = cursor.fetchone()

    # Close the database connection
    conn.close()

    # Return the abstract if a record was found, else return None
    return result[0] if result else None

# Example usage
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/53BP1_RIF1.db'  # Replace with your database path
article_id = 3  # The ID of the article
summary = get_abstract_by_id(db_path, article_id)

if summary:
    summary = summarize_text(summary)
    print("Summary:", summary)
else:
    print("No record found for the given ID.")


Summary:  1. RIF1, inhibit, UFB resolution; 2. RIF1, localize on PICH-coated, not RPA-coated UFBs; 3. RIF1 de...


#### Exporting all db to csv file

In [5]:
import sqlite3
import csv

def export_db_to_csv(db_path, csv_path):
    """
    Export data from an SQLite database table to a CSV file.
    :param db_path: Path to the SQLite database file.
    :param csv_path: Path to the CSV file where data will be written.
    """
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Prepare SQL query to select all data from the articles table
    query = "SELECT * FROM articles"

    # Execute the query
    cursor.execute(query)

    # Fetch all records
    records = cursor.fetchall()

    # Get the column headers
    columns = [description[0] for description in cursor.description]

    # Open a CSV file for writing
    with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)

        # Write the column headers
        writer.writerow(columns)

        # Write the records
        writer.writerows(records)

    # Close the database connection
    conn.close()

    print(f"Data exported successfully to {csv_path}")

# Example usage
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/53BP1_RIF1.db'  # Replace with your database path
csv_path = '/Users/probio/Documents/codes/Publication2Knowledge/53BP1_RIF1.csv'  # Replace with your desired CSV path
export_db_to_csv(db_path, csv_path)


Data exported successfully to /Users/probio/Documents/codes/Publication2Knowledge/53BP1_RIF1.csv


### Combining multiple dataset

Searching keywords, turning them into summaries, and saving them into a database is a compute-consuming process; therefore, it is essential to search for each keyword independently. As a consequence, we are ending up with multiple databases that need to be integrated. Since databases has the same format and name of all databases start with "publication-" and end with ".db", can you write an script to combine them in python.

In [20]:
def merge_databases_with_logging(source_dir, merged_db_name):
    """
    Merges multiple SQLite databases with a similar schema into a new database with logging.

    :param source_dir: Directory containing the source databases.
    :param merged_db_name: Name of the merged database.
    :return: None
    """
    # File pattern for matching database files
    db_pattern = os.path.join(source_dir, "publication_*.db")

    # Find all database files in the directory
    db_files = glob.glob(db_pattern)
    print(f"Found {len(db_files)} database files.")

    # Create or connect to the new merged database
    merged_conn = sqlite3.connect(merged_db_name)
    merged_cursor = merged_conn.cursor()

    # Create a table in the merged database (excluding the AUTOINCREMENT from the ID)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS articles (
        id INTEGER PRIMARY KEY,
        gene TEXT,
        title TEXT,
        abstract TEXT,
        authors TEXT,
        journal TEXT,
        year TEXT,
        keywords TEXT,
        summary TEXT
    )
    """
    merged_cursor.execute(create_table_query)

    # Iterate over each database file and insert data into the merged database
    for db_file in db_files:
        print(f"Processing file: {db_file}")
        source_conn = sqlite3.connect(db_file)
        source_cursor = source_conn.cursor()

        # Select data excluding the ID
        select_query = "SELECT gene, title, abstract, authors, journal, year, keywords, summary FROM articles"
        rows_inserted = 0
        for row in source_cursor.execute(select_query):
            # Insert data into the merged database
            insert_query = "INSERT INTO articles (gene, title, abstract, authors, journal, year, keywords, summary) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
            merged_cursor.execute(insert_query, row)
            rows_inserted += 1

        print(f"Inserted {rows_inserted} rows from {db_file}")
        # Close the source database connection
        source_conn.close()

    # Commit changes and close the merged database connection
    merged_conn.commit()
    merged_conn.close()

# Example usage
source_directory = '/Users/probio/Documents/codes/Publication2Knowledge/'  # Replace with the actual directory path
merged_db_name = '/Users/probio/Documents/codes/Publication2Knowledge/merged_database.db'  # Replace with the desired path for the merged database

# Uncomment the following line to run the script with logging
merge_databases_with_logging(source_directory, merged_db_name)


Found 4 database files.
Processing file: /Users/probio/Documents/codes/Publication2Knowledge/publication_BRCA1_BRD4.db
Inserted 11 rows from /Users/probio/Documents/codes/Publication2Knowledge/publication_BRCA1_BRD4.db
Processing file: /Users/probio/Documents/codes/Publication2Knowledge/publication_G9a_GLP.db
Inserted 14 rows from /Users/probio/Documents/codes/Publication2Knowledge/publication_G9a_GLP.db
Processing file: /Users/probio/Documents/codes/Publication2Knowledge/publication_53BP1_RIF1.db
Inserted 20 rows from /Users/probio/Documents/codes/Publication2Knowledge/publication_53BP1_RIF1.db
Processing file: /Users/probio/Documents/codes/Publication2Knowledge/publication_MGA_MYC.db
Inserted 13 rows from /Users/probio/Documents/codes/Publication2Knowledge/publication_MGA_MYC.db


### Cleaning summary and turning it into knowledge

First step is recovering summaries from databases. The idea is first explore and find the way by just taking one summary from database. The pulled summary is inspected to find the way for cleaning and turining it into a format that can be used later on with NetworkX. NetworkX uses the provided data to create a knowledge graph. A cumulated data in the format of knowledge graph is used to identify the function of a gene or a group of genes to compete with gene ontology analysis.

In [38]:
# Pulling out a summary from database
import sqlite3

def summarize_text(text, max_length=100):
    """
    Generate a simple summary of the text by truncating it.
    :param text: The text to summarize.
    :param max_length: The maximum length of the summary.
    :return: A summarized version of the text.
    """
    return text if len(text) <= max_length else text[:max_length] + '...'

def get_abstract_by_id(db_path, article_id):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Prepare and execute the SQL query
    query = f"SELECT summary FROM articles WHERE id = {article_id}"
    cursor.execute(query)

    # Fetch the first record
    result = cursor.fetchone()

    # Close the database connection
    conn.close()

    # Return the abstract if a record was found, else return None
    return result[0] if result else None

# Example usage
db_path = '/Users/probio/Documents/codes/Publication2Knowledge/merged_database.db'  # Replace with your database path
article_id = 20  # The ID of the article
summary = get_abstract_by_id(db_path, article_id)

if summary:
    summary = summarize_text(summary)
    print("Summary:", summary)
else:
    print("No record found for the given ID.")


Summary: N-WASP ---> Inhibits ---> p53-induced Senescence (via G9a/GLP, DNMT1). 7,12-Dimethylbenz(a)anthracen...


### Sending summary to LLM for clane up.

Prompt:

"You are a scientist and your job is to clean up and correct information that you recive. Information are in the format of graphical summary plus a short text that define context of informaton. In the graphical summary "--->" is signe of activation or any words that carry a mean of positive effect. Additionally, "---|" is signe of inhibition and any words that carry a mean of negative effect. Your job is to clean up summary by fomating interaction that you fine. For example "BAP1--->BRD4 (via MYC, BET inhibitors in BAP1 deficient context)." can clean up as follow. BAP1, activate, BRD4"

"You are a scientist, and your job is to clean up and correct information that you receive. Information is in the format of a graphical summary plus a short text that defines the context of the information. In the graphical summary ---> is a sign of activation or any words that carry a positive effect. Additionally, ---| is a sign of inhibition, as are any words that carry a negative effect. For example, if you read 'A--->B inhibition' then you can change it to 'A---|B'. Or lean a wrong sign."

In [39]:
from openai import OpenAI

client= OpenAI(base_url="http://localhost:5001/v1", api_key="not-needed")

try:
    completion = client.chat.completions.create(
        model="marcoroni",  # Ensure the model name is correct. It should be a valid model name from OpenAI.
        messages=[
            {
                "role": "system", 
                "content": ("You are a scientist, and your job is to clean up and correct information that you receive. Information is in the format of a graphical summary plus a short text that defines the context of the information. In the graphical summary ---> is a sign of activation or any words that carry a positive effect. Additionally, ---| is a sign of inhibition, as are any words that carry a negative effect. Your job is to identify relations between component and check if signs are used corrctly. If not you should corrct them.")
             },
            {
                "role": "user", 
                "content": (summary)
            },
         ],
        temperature=0.4,
    )
except Exception as e:
    print(f"An error occurred: {e}")
    # Handle the error appropriately

# Ensure that 'text' contains the string content of the message
text = completion.choices[0].message.content if completion else ""
print(text)

 N-WASP --> Inhibits --> P53-Induced Senescence (via G9a/GLP, DNMT1); Misleading arrow direction, should be "N-WASP inhibits..." with a correct sign for inhibition '-->' replaced by '-->'.
